In [54]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy import stats
import matplotlib.pyplot as plt
import os
import math
plt.rcParams.update({'font.size': 20})

In [55]:
search_name = "midas_oracle"
# search_name = "midasB_oracle"

In [56]:
ss1 = [0.0, 0.25 ,0.5, 1.0, 2.0]
ss2 = [-1.0, -0.5, -0.25, 0.0, 0.25 ,0.5, 1.0]

searching = {
    "midas_oracle": ["es/add_global_deg_min_%.4f" % (alpha) for alpha in ss1],
    "midasB_oracle": ["essz/add_global_deg_min_%.2f_%.2f" % (alpha, beta) for alpha in ss1 for beta in ss2],
}

In [57]:
evallist = ["clusteringcoef", "densification", "effdiameter", "overlapness"]
evaldistlist = ["degree", "intersection", "pairdeg", "size", "sv", "wcc"]

In [58]:
dataset = ["email-Enron-full", "email-Eu-full",  
           "contact-high-school", "contact-primary-school",
          "NDC-classes-full", "NDC-substances-full", "tags-ask-ubuntu", "tags-math-sx", 
           "threads-ask-ubuntu", "coauth-MAG-History-full", "coauth-MAG-Geology-full"]

In [60]:
bestalgo = {}
for dataname in dataset:
    result = {}
    
    d = pd.read_csv("csvs/%s/eval_diff_norm_all_%s.txt" % (dataname, search_name))
    minnorm = min(d["avg"])
    maxnorm = max(d["avg"])
    for i, row in d.iterrows():
        algoname = row["algoname"]
        norm = (row["avg"] - minnorm) / (maxnorm - minnorm)
        result[algoname] = norm
    
    
    d = pd.read_csv("csvs/%s/eval_diff_rank_all_%s.txt" % (dataname, search_name))
    minrank = min(d["avg"])
    maxrank = max(d["avg"])
    for i, row in d.iterrows():
        algoname = row["algoname"]
        rank = (row["avg"] - minrank) / (maxrank - minrank)
        result[algoname] += rank
    
    
    bestalgoname = ""
    bestalgoval = 1000
    for k, v in result.items():
        if bestalgoval > v:
            bestalgoname = k
            bestalgoval = v
    bestalgo[dataname] = bestalgoname

In [62]:
name = search_name
dirpath = "../results/" + name + "/"

if os.path.isdir(dirpath) is False:
    os.makedirs(dirpath)
if os.path.isfile(dirpath + "search_result.txt"):
    os.remove(dirpath + "search_result.txt")

for dataname in dataset:
    with open(dirpath + "search_result.txt", "a") as f:
        algoname = bestalgo[dataname]
        line = "/home/MiDaS-B/results/{}/{}/".format(algoname, dataname) + "\n"
        f.write(line)